# Tema 8: Ficheros y módulos (IV)
## Guardar información entre ejecuciones
Ahora que sabemos leer y escribir en ficheros e importar módulos, tenemos todo lo necesario para conservar la información entre sesiones de ejecución de un mismo programa.

En este cuaderno simplemente vamos ver cómo se escribe un programa para gestionar una colección de libros, que cree un archivo por cada libro que queramos guardar, y que nos permita, a través de un menú, modificar esta información en el archivo y consultarla. El código es muy parecido al del cuaderno 0704 para consultar colecciones de películas y libros.

Primero tenemos que pensar qué acciones vamos a permitir al usuario. Por ejemplo: guardar un nuevo libro, modificar los datos de un libro ya existente y consultar los datos de un libro. Además tendremos que dar la opción de salir, y crear en nuestro código un último caso de _fallback_ para cuando el usuario no escriba ninguna de las opciones que hemos pensado para él. Es decir, que lo primero que tenemos que pensar es el esqueleto del programa, lo que terminará siendo el programa principal, y luego escribiremos las funciones. En este caso, quedaría así:

In [ ]:
# Menú
continuar = True
while continuar:
    opcion = int(input("""¿Qué quieres hacer?
                       1. Añadir un libro.
                       2. Modificar los datos de un libro.
                       3. Consultar los datos de un libro.
                       4. Salir.
                       """))
    
    if opcion == 1:
        nuevo_libro()

    elif opcion == 2:
        modificar_libro()

    elif opcion == 3:
        consultar_libro()
    
    elif opcion == 4:
        print("¡Adiós!")
        continuar = False
    
    else:
        print("Debes escribir un número entre 1 y 4.")

Ahora tenemos que pensar qué deben hacer nuestras funciones `nuevo_libro()`, `modificar_libro()` y `consultar_libro()`.

Las dos primeras van a ser muy parecidas:
- pedirle los datos del libro al usuario
- crear/sobreescribir un archivo con los datos
- imprimir un mensaje de confirmación:
    - en el caso de `nuevo_libro()`, dirá algo así como `Se ha añadido 'Título' a la biblioteca`
    - en el caso de `modificar_libro()`, dirá `Se ha actualizado la información sobre 'Título' en la biblioteca`
    
Solamente difieren en el mensaje de confirmación y todo lo demás es igual, lo que significa que podremos reutilizar mucho código. Así que vamos a escribir 4 subfunciones: una para pedir los datos al usuario, otra para guardarlos en un archivo (que nos da igual si existe previamente o no, así que nos vale tanto para `nuevo_libro` como para `modificar_libro()`) y otras dos para imprimir los dos tipos de mensaje de confirmación.

Entonces, primero, ¿qué datos vamos a pedir? Podemos pedir, por ejemplo, el título, el autor, el año de publicación y el género. Además, tenemos que pensar en cómo vamos a llamar a nuestros archivos. Podemos ponerles, de nombre, el título del libro, porque será una string y para nuestros propósitos seguro que es el dato más identificativo del libro, y de extensión ".libro". Veamos cómo queda todo esto en funciones:

In [1]:
def pedir_datos():
    # Pide los datos del libro al usuario
    titulo = input("Título: ")
    autor = input("Autor(es): ")
    anno = input("Año de publicación: ")
    genero = input("Género: ")
    return titulo, autor, anno, genero

def guardar_datos(titulo, autor, anno, genero):
    # Guarda los datos del libro en un archivo
    archivo = open(titulo + ".libro", "w", encoding = "UTF-8")
    archivo.write(titulo + "\n" + autor + "\n" + anno + "\n" + genero + "\n")
    archivo.close()

titulo, autor, anno, genero = pedir_datos()
guardar_datos(titulo, autor, anno, genero)

Título: El perfume
Autor(es): Patrick Süskind
Año de publicación: 1985
Género: Novela


Parece que todo ha ido bien. Además, nos ha creado el fichero `El perfume.libro` con las líneas esperadas, así que definitivamente el código está bien.

Fíjate en la gran diferencia que hay entre las dos funciones: en `pedir_datos()` no podemos definir ningún parámetro; precisamente lo que estamos haciendo es pedir los datos al usuario para devolverlos, así que otra cosa que tenemos que hacer es escribir una línea de `return` con todos esos datos. En cambio, en `guardar_datos()` es al contrario: ya tenemos los datos y lo que queremos es guardarlos en un archivo, pero no nos interesa que la función devuelva nada, porque una vez que están guardados los datos en el archivo ya ha hecho todo lo que queríamos.

Es importante que siempre mantengamos el mismo orden en los datos: si el `return` de `pedir_datos()` es `titulo, autor, anno, genero`, los parámetros de `guardar_datos()` también deben ir en ese orden (aunque no tienen por qué llamarse igual, porque son funciones distintas). 

Y ahora, vamos a definir las distintas funciones que imprimen los mensajes de confirmación. Podemos pasarle el parámetro `titulo` para que salga un mensaje más personalizado:

In [3]:
def imprimir_confirmacion_guardado(titulo):
    # Imprime el mensaje de confirmación de guardado
    print("Ok, se ha añadido", titulo, "a la biblioteca.")
    print()

def imprimir_confirmacion_actualizacion(titulo):
    # Imprime el mensaje de confirmación de actualización
    print("Ok, se ha actualizado la información sobre", titulo, "en la biblioteca.")
    print()

Bien, ya tenemos todo lo necesario para definir nuestras funciones `nuevo_libro()` y `modificar_libro()`:

In [4]:
def nuevo_libro():
    # Pide y guarda los datos en el archivo correspondiente
    titulo, autor, anno, genero = pedir_datos()
    guardar_datos(titulo, autor, anno, genero)
    imprimir_confirmacion_guardado(titulo)

def modificar_libro():
    # Pide y guarda los datos en el archivo correspondiente
    titulo, autor, anno, genero = pedir_datos()
    guardar_datos(titulo, autor, anno, genero)
    imprimir_confirmacion_actualizacion(titulo)
    
nuevo_libro()
modificar_libro()

Título: El hobbit
Autor(es): J. R. R. Tolkien
Año de publicación: 1937
Género: Novela
Ok, se ha añadido El hobbit a la biblioteca.

Título: Edipo rey
Autor(es): Sófocles
Año de publicación: 430 a. C.
Género: Teatro
Ok, se ha actualizado la información sobre Edipo rey en la biblioteca.



Perfecto. Fíjate en que, como `pedir_datos()` devuelve 4 valores, tenemos que llamar a esta función previendo que esos valores tienen que guardarse en otras tantas variables. Y, al contrario, en la siguiente línea llamamos a `guardar_datos()` sin meter su salida en ninguna variable, pero sí pasándoles como parámetros las variables que contienen los datos que acabamos de obtener.

Ahora tenemos que definir `consultar_libro()`. Estaría bien que, ya que vamos a tener que abrir archivos en modo lectura, nos aseguremos antes de ello de que existe el archivo correspondiente al libro que nos pide el usuario.

Después, sabemos que los archivos tienen la información guardada siguiendo una determinada estructura, que es la siguiente: título, autor, año y género, cada uno en una línea. Así que una cosa que podemos hacer es usar `readlines()` sobre el archivo para meter las líneas en una lista, y después imprimir de forma bonita cada elemento de la lista.

Además, es probable que el usuario quiera consultar más de un libro, así que vamos a darle esa opción antes de devolverle al menú principal. Y tendremos que gestionar los inputs erróneos (de libros de los que no tenemos un archivo) del usuario, haciendo que el programa le vuelva a preguntar hasta que dé uno correcto. Veamos cómo queda:

In [7]:
import os

def consultar_libro():
    # Consulta los datos en los archivos y los imprime
    continuar = True
    while continuar:
        titulo = input("Título del libro que quieres consultar: ")
        if os.path.isfile(titulo + ".libro"): # comprobamos que existe el archivo
            archivo = open(titulo + ".libro", "r", encoding = "UTF-8")
            lista = archivo.readlines()
            titulo = lista[0].strip()
            autor = lista[1].strip()
            anno = lista[2].strip()
            genero = lista[3].strip()
            print("Título:", titulo)
            print("Autor(es):", autor)
            print("Año de publicación:", anno)
            print("Género:", genero)
            print()
        else: # si no existe el archivo, preguntamos hasta que dé un nombre válido
            print("No tienes ningún libro en tu biblioteca llamado", titulo)
        
        otro_libro = input("¿Quieres consultar otro libro?: ") # preguntamos si quiere seguir consultando libros
        if otro_libro.lower() == "n" or otro_libro.lower() == "no":
            continuar = False

consultar_libro()

Título del libro que quieres consultar: El hobbit
Título: El hobbit
Autor(es): J. R. R. Tolkien
Año de publicación: 1937
Género: Novela

¿Quieres consultar otro libro?: sí
Título del libro que quieres consultar: El perfume
Título: El perfume
Autor(es): Patrick Süskind
Año de publicación: 1985
Género: Novela

¿Quieres consultar otro libro?: no


¡Ya lo tenemos todo! Si pegamos todo esto, nos queda el siguiente código:

In [ ]:
### Programa para gestionar los libros de una biblioteca

import os

### FUNCIONES

def pedir_datos():
    # Pide los datos del libro al usuario
    titulo = input("Título: ")
    autor = input("Autor(es): ")
    anno = input("Año de publicación: ")
    genero = input("Género: ")
    return titulo, autor, anno, genero

def guardar_datos(titulo, autor, anno, genero):
    # Guarda los datos del libro en un archivo
    archivo = open(titulo + ".libro", "w", encoding = "UTF-8")
    archivo.write(titulo + "\n" + autor + "\n" + anno + "\n" + genero + "\n")
    archivo.close()

def imprimir_confirmacion_guardado(titulo):
    # Imprime el mensaje de confirmación de guardado
    print("Ok, se ha añadido", titulo, "a la biblioteca.")
    print()

def imprimir_confirmacion_actualizacion(titulo):
    # Imprime el mensaje de confirmación de actualización
    print("Ok, se ha actualizado la información sobre", titulo, "en la biblioteca.")
    print()    

def nuevo_libro():
    # Pide y guarda los datos en el archivo correspondiente
    titulo, autor, anno, genero = pedir_datos()
    guardar_datos(titulo, autor, anno, genero)
    imprimir_confirmacion_guardado(titulo)

def modificar_libro():
    # Pide y guarda los datos en el archivo correspondiente
    titulo, autor, anno, genero = pedir_datos()
    guardar_datos(titulo, autor, anno, genero)
    imprimir_confirmacion_actualizacion(titulo)
    
def consultar_libro():
    # Consulta los datos en los archivos y los imprime
    continuar = True
    while continuar:
        titulo = input("Título del libro que quieres consultar: ")
        if os.path.isfile(titulo + ".libro"):
            archivo = open(titulo + ".libro", "r", encoding = "UTF-8")
            lista = archivo.readlines()
            titulo = lista[0].strip()
            autor = lista[1].strip()
            anno = lista[2].strip()
            genero = lista[3].strip()
            print("Título:", titulo)
            print("Autor(es):", autor)
            print("Año de publicación:", anno)
            print("Género:", genero)
            print()
        else:
            print("No tienes ningún libro en tu biblioteca llamado", titulo)
        otro_libro = input("¿Quieres consultar otro libro?: ")
        if otro_libro.lower() == "n" or otro_libro.lower() == "no":
            continuar = False

### PROGRAMA PRINCIPAL

# Menú
continuar = True
while continuar:
    opcion = int(input("""¿Qué quieres hacer?
                       1. Añadir un libro.
                       2. Modificar los datos de un libro.
                       3. Consultar los datos de un libro.
                       4. Salir.
                       """))
    
    # Opción 1. Añadir un libro
    if opcion == 1:
        nuevo_libro()

    # Opción 2. Modificar un libro
    elif opcion == 2:
        modificar_libro()

    # Opción 3. Consultar un libro
    elif opcion == 3:
        consultar_libro()
    
    # Opción 4. Salir
    elif opcion == 4:
        print("¡Adiós!")
        continuar = False
    
    # Fallback
    else:
        print("Debes escribir un número entre 1 y 4.")


Y si guardamos la parte de las funciones en un archivo que llamemos `biblioteca.py`, podemos importarlo y reducir nuestro programa a las siguientes líneas:

In [ ]:
### Programa para gestionar los libros de una biblioteca

from biblioteca import nuevo_libro, modificar_libro, consultar_libro

# Menú
continuar = True
while continuar:
    opcion = int(input("""¿Qué quieres hacer?
                       1. Añadir un libro.
                       2. Modificar los datos de un libro.
                       3. Consultar los datos de un libro.
                       4. Salir.
                       """))
    
    # Opción 1. Añadir un libro
    if opcion == 1:
        nuevo_libro()

    # Opción 2. Modificar un libro
    elif opcion == 2:
        modificar_libro()

    # Opción 3. Consultar un libro
    elif opcion == 3:
        consultar_libro()
    
    # Opción 4. Salir
    elif opcion == 4:
        print("¡Adiós!")
        continuar = False
    
    # Fallback
    else:
        print("Debes escribir un número entre 1 y 4.")

Como ves, hemos tenido que tomar muchas decisiones que no tienen mucho que ver con la parte técnica de la programación, sino con cómo es mejor gestionar los datos, qué tiene más sentido, cuál va a ser el alcance y el propósito de nuestro programa... Esto también se aprende con la práctica. Al programar, pasas más tiempo diseñando lo que quieres construir, haciendo y deshaciendo el código, probándolo, buscando y leyendo documentación en internet sobre el lenguaje o la librería que estás usando... que escribiendo código realmente. Es decir, [no se parece nada a cómo nos lo pintan en las películas](https://www.youtube.com/watch?v=HluANRwPyNo).

## Ejercicios
### 080401
Modifica el programa anterior incluyendo una nueva opción: registrar un préstamo. Tendrás que escribir una función parecida a `consultar_libro()`, que compruebe si el libro está en la biblioteca (es decir, que existe el archivo correspondiente al libro que se quiere prestar) y, en caso afirmativo, solamente añada un par de líneas: el nombre de la persona a la que se lo prestamos, y la fecha de devolución (como si fuéramos una biblioteca de verdad). Recuerda: tendrás que abrir el archivo en modo _append_, o si no lo que estarás haciendo es sobreescribir todo el archivo.